In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import re
import numpy as np
import warnings

warnings.filterwarnings('ignore')

train_df = pd.read_csv('train_text.csv')
train_df2 = pd.read_csv('test_text.csv')
train_label = pd.read_csv('test1.csv')
train_answer = pd.read_csv('answer_train.csv')
lb = LabelEncoder()
train_answer['label'] = lb.fit_transform(train_answer['label'])
train_label['label'] = lb.transform(train_label['label'])
train_label.columns = ['id', 'label']
train_df2 = pd.merge(train_df2, train_label, on='id', how='left')
train_df = pd.concat([train_df, train_df2], axis=0)
test_df = pd.read_csv('test_text2.csv')

In [12]:
# preprocess
pattern = re.compile("[^\u4e00-\u9fa5]")
def preprocess_train(x):
    text = str(x['text'])
    text = re.sub(pattern, '', text)
    text = ''.join(text.split())
    if '无访问权限' in text:
        return ''
    elif len(text) > 5000:
        return text[:5000]
    else:
        return text
    
train_df['text_new'] = train_df.apply(lambda x: preprocess_train(x), axis=1)
train_df = train_df[train_df['text_new']!=''].reset_index(drop=True)[['id', 'text_new','label']]
def preprocess_test(x):
    text = str(x['text'])
    text = re.sub(pattern, '', text)
    text = ''.join(text.split())
    title = re.sub(pattern, '', x['id'])
    if len(title) != 0:
        return ''       
    else:
        if '无访问权限' in text:
            return ''
        elif len(text) > 5000:
            return text[:5000]
        else:
            return text
test_df['text_new'] = test_df.apply(lambda x: preprocess_test(x), axis=1)
test_df = test_df[test_df['text_new']!=''].reset_index(drop=True)[['id', 'text_new']]

In [7]:
train_set, valid_set = train_test_split(train_df, stratify=train_df['label'], test_size=0.2, random_state=2020)
train_set[['label', 'text_new']].to_csv('bert/data_content/train.tsv', sep='\t', index=False, header=None)
valid_set[['label', 'text_new']].to_csv('bert/data_content/dev.tsv', sep='\t', index=False, header=None)
test_df['label'] = 0
test_df[['label', 'text_new']].to_csv('bert/data_content/test.tsv', sep='\t', index=False, header=None)

In [14]:
# predict
test_predict = pd.read_csv('bert/output_content/test_results.tsv', sep='\t', header=None)
test_label = lb.inverse_transform([np.argmax(i) for i in test_predict.values])
test_df['label'] = test_label
test_df[['id', 'label']].to_csv('content_final.csv', index=False)